# Preparação da base de dados para treinar os classificadores

In [1]:
from src import (
    Folder,
    SegModel,
    SegmentedList,
    segmentation_fails,
    ds2df,
    save_results,
    File2df,
    Best_rf,
    Rf,
    MLP,
    SVM,
    XGBoost,
    KNN,
    GBM,
    List2Best,
    Save_Model_PKL
    )

from os.path import join
DATA = "Dados"
MODEL = "Models"

In [2]:
#adquindo as imagens e seus respectivos rotulos
dataBasePath = join(DATA,"dados")
#dataBasePath = join(DATA,"changed","rotacionados")
#dataBasePath = join(DATA,"changed","segmentation")
images, labels = Folder(dataBasePath)


In [3]:
doente = 0
bom = 0

for famacha in labels:
    grau = int(famacha.split('.')[-2][-1])
    if grau > 2:
        doente +=1
    else:
        bom +=1
        
print(f"Dos {len(labels)} animais:\nDoentes: {doente}\nSaudaveis: {bom}")

Dos 323 animais:
Doentes: 188
Saudaveis: 135


## Caso as imagens não estejam segmentadas, podemos segmentalas

In [4]:

segmentation_model = SegModel(join(MODEL,"best.pt"))
process_images = SegmentedList(images,segmentation_model,is_resized=False) # Por padrão redimensionar garante menor taxa de falha
#process_images = SegmentedList(images,segmentation_model,is_resized=True)

WARNING ⚠️ 'boxes' is deprecated and will be removed in 'ultralytics 8.299999999999999' in the future. Please use 'show_boxes' instead.

0: 640x640 1 eye_membrane, 10.6ms
Speed: 4.0ms preprocess, 10.6ms inference, 391.2ms postprocess per image at shape (1, 3, 640, 640)
WARNING ⚠️ 'boxes' is deprecated and will be removed in 'ultralytics 8.299999999999999' in the future. Please use 'show_boxes' instead.

0: 640x640 1 eye_membrane, 9.8ms
Speed: 3.3ms preprocess, 9.8ms inference, 3.0ms postprocess per image at shape (1, 3, 640, 640)
WARNING ⚠️ 'boxes' is deprecated and will be removed in 'ultralytics 8.299999999999999' in the future. Please use 'show_boxes' instead.

0: 640x640 1 eye_membrane, 8.1ms
Speed: 3.7ms preprocess, 8.1ms inference, 2.4ms postprocess per image at shape (1, 3, 640, 640)
WARNING ⚠️ 'boxes' is deprecated and will be removed in 'ultralytics 8.299999999999999' in the future. Please use 'show_boxes' instead.

0: 640x640 (no detections), 9.8ms
Speed: 3.1ms preprocess, 9.

### com as imagens processadas, precisamos descartar as imagens que não foram segmentadas e salvar seu nome em um arquivo com as falhas

In [5]:
fails = segmentation_fails(process_images,labels)

#print(f"Quantidade: {len(fails)}\nFalhas:",fails)
#print("% = ",len(fails)/len(images))

#msg = "falhas-normais-"
#msg = "falhas-redimensionadas-"


### Podemos salvar as falhas em um arquivo de texto para vizualizar depois

In [ ]:
"""
anotation_fails= join("docs","Falhas","teste.txt")

with open(anotation_fails, "w") as arquivo:
        for fail in fails:
            arquivo.write(fail + "\n")
"""

### fazendo a filtragem das imagens válidas

In [6]:
valid_images = []
valid_labels = []

for i in range(0,len(process_images),1):
    if labels[i] not in fails:
        valid_images.append(process_images[i])
        valid_labels.append(labels[i])
        

#print(len(valid_labels),len(valid_images))

# Extraindo as Caracteristicas de interesse da base de dados

In [7]:
#df = ds2df(process_images,labels)
#df = ds2df(valid_images,valid_labels)
df = ds2df(images,labels)

### Visando evitar repetir o processamento para a mesma base de dados podemos exportar a mesma para um arquivo e apenas ler a partir do arquivo quando quisermos novamente

In [8]:

save_results(df,DATA,"Normal2",".csv")

### Carregando nossos dados a partir de um dataset .csv

In [ ]:
# caso não tenhamos nosso DataFrame carregado, podemos carregalo apartir de um arquivo
df = File2df(join(DATA,"rotacionadas.csv"))

#df.drop("Unnamed: 0",axis=1,inplace=True)
#df.columns

# Treinando os classificadores

In [9]:
rf = Best_rf(df,join("docs","classification"))


In [ ]:
Save_Model_PKL(rf,join(MODEL,"RF"))

### Vamos pegar a melhor versão de cada modelo

In [10]:
bests_list = []

funcs = [Rf, MLP,SVM,XGBoost,KNN,GBM]
names = ["RF","MLP","SVM","XGBoost","KNN","GBM"]

args = (0.1,0.2,0.3)

for i,func in enumerate(funcs,0):

    for arg in args:
        
        model, accuracy, precision, recall, f1, kappa = func(df,arg)
        bests_list.append((names[i],arg,accuracy, precision, recall, f1, kappa))
    
bests_list

[('RF', 0.1, 60.61, 64.71, 61.11, 62.86, 20.99),
 ('RF', 0.2, 66.15, 72.22, 68.42, 70.27, 31.05),
 ('RF', 0.3, 65.98, 68.97, 72.73, 70.8, 30.12),
 ('MLP', 0.1, 57.58, 60.0, 66.67, 63.16, 13.48),
 ('MLP', 0.2, 64.62, 72.73, 63.16, 67.61, 29.05),
 ('MLP', 0.3, 65.98, 76.19, 58.18, 65.98, 33.16),
 ('SVM', 0.1, 69.7, 70.0, 77.78, 73.68, 38.2),
 ('SVM', 0.2, 69.23, 70.45, 81.58, 75.61, 34.54),
 ('SVM', 0.3, 73.2, 70.42, 90.91, 79.37, 42.84),
 ('XGBoost', 0.1, 66.67, 68.42, 72.22, 70.27, 32.4),
 ('XGBoost', 0.2, 67.69, 72.97, 71.05, 72.0, 33.83),
 ('XGBoost', 0.3, 68.04, 68.75, 80.0, 73.95, 33.22),
 ('KNN', 0.1, 63.64, 65.0, 72.22, 68.42, 25.84),
 ('KNN', 0.2, 67.69, 70.73, 76.32, 73.42, 32.39),
 ('KNN', 0.3, 67.01, 68.25, 78.18, 72.88, 31.27),
 ('GBM', 0.1, 63.64, 66.67, 66.67, 66.67, 26.67),
 ('GBM', 0.2, 60.0, 65.0, 68.42, 66.67, 16.75),
 ('GBM', 0.3, 68.04, 69.35, 78.18, 73.5, 33.61)]

In [ ]:
output = join("docs","classification")

In [ ]:
#best = List2Best(bests_list,output,f"{len(valid_images)}.csv",True)
best = List2Best(bests_list,output,f"F{len(images)}.csv",True)
best

# caso gostemos do resultado e desejemos salvar o modelo

In [ ]:
#Save_model(model,join(MODEL,best))

# Avaliando o comparativo dos modelos 

In [ ]:
# caso não tenhamos nosso DataFrame carregado, podemos carregalo apartir de um arquivo
result_models = File2df(join(output,"not rf.csv"))

#result_models.drop("Unnamed: 0",axis=1,inplace=True)
result_models.columns

In [ ]:
result_models

In [ ]:
rotulos = result_models.Modelo.unique()
rotulos

In [ ]:
# Suponha que 'df' seja o DataFrame contendo os resultados dos modelos

# Rótulos dos modelos
model_labels = ["RF","MLP","SVM","XGBoost","KNN","GBM"]

# Lista para armazenar os melhores resultados de cada modelo
best_results = []

# Iterar sobre os rótulos dos modelos
for model_label in model_labels:
    # Filtrar o DataFrame para o modelo atual
    model_df = result_models[result_models['Modelo'] == model_label]
    
    # Encontrar a linha com o melhor resultado para o modelo atual
    best_row = model_df.loc[model_df['Total'].idxmax()]
    
    # Adicionar o resultado à lista de melhores resultados
    best_results.append(best_row)

import pandas as pd
# Criar um DataFrame com os melhores resultados
print(best_results)
best_results_df = pd.DataFrame(best_results)

# Exibir o DataFrame com os melhores resultados
print(best_results_df)
